# Unet 3D (Baseline)

In [1]:
from utils import instantiate_generator

train_data_pipeline, val_data_pipeline = instantiate_generator(train_size=0.8, batch_size = 2)

2022-10-30 17:44:51.651997: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-30 17:44:53.014605: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-30 17:44:53.014647: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-30 17:44:53.187619: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-30 17:44:55.855317: W tensorflow/stream_executor/platform/de

Dataset found on device at : './challenge_dataset/.'


2022-10-30 17:45:08.179039: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-30 17:45:08.179700: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-30 17:45:08.179725: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-IN2P6PG): /proc/driver/nvidia/version does not exist
2022-10-30 17:45:08.183954: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from keras.layers import BatchNormalization, Conv3D, Cropping3D, ZeroPadding3D, Activation, Input
from keras.layers import MaxPooling3D, UpSampling3D, Conv3DTranspose, concatenate
from keras.models import Model


# input shape without batch size and we add one channel for BW color so dim=4
in_layer = Input((None, None, None, 1)) 
bn = BatchNormalization()(in_layer)
cn1 = Conv3D(8, 
             kernel_size = (1, 5, 5), 
             padding = 'same',
             activation = 'relu')(bn)
cn2 = Conv3D(8, 
             kernel_size = (3, 3, 3),
             padding = 'same',
             activation = 'linear')(cn1)
bn2 = Activation('relu')(BatchNormalization()(cn2))

dn1 = MaxPooling3D((2, 2, 2))(bn2)
cn3 = Conv3D(16, 
             kernel_size = (3, 3, 3),
             padding = 'same',
             activation = 'linear')(dn1)
bn3 = Activation('relu')(BatchNormalization()(cn3))

dn2 = MaxPooling3D((1, 2, 2))(bn3)
cn4 = Conv3D(32, 
             kernel_size = (3, 3, 3),
             padding = 'same',
             activation = 'linear')(dn2)
bn4 = Activation('relu')(BatchNormalization()(cn4))

up1 = Conv3DTranspose(16, 
                      kernel_size = (3, 3, 3),
                      strides = (1, 2, 2),
                     padding = 'same')(bn4)

cat1 = concatenate([up1, bn3])

up2 = Conv3DTranspose(8, 
                      kernel_size = (3, 3, 3),
                      strides = (2, 2, 2),
                     padding = 'same')(cat1)

pre_out = concatenate([up2, bn2])

pre_out = Conv3D(1, 
             kernel_size = (1, 1, 1), 
             padding = 'same',
             activation = 'sigmoid')(pre_out)

pre_out = Cropping3D((1, 2, 2))(pre_out) # avoid skewing boundaries
out = ZeroPadding3D((1, 2, 2))(pre_out)
sim_model = Model(inputs = [in_layer], outputs = [out])
sim_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 None, 1)]                                                        
                                                                                                  
 batch_normalization (BatchNorm  (None, None, None,   4          ['input_1[0][0]']                
 alization)                     None, 1)                                                          
                                                                                                  
 conv3d (Conv3D)                (None, None, None,   208         ['batch_normalization[0][0]']    
                                None, 8)                                                      

In [3]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

sim_model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['binary_accuracy', 'mse'])

weight_path="{}_weights.best.hdf5".format('model')
checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=5, verbose=1, mode='auto', epsilon=0.0001, cooldown=5, min_lr=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=10) # probably needs to be more patient, but kaggle time is limited

callbacks_list = [checkpoint, early, reduceLROnPlat]

In [4]:
sim_model.fit(train_data_pipeline, 
            epochs=20,
            validation_data=val_data_pipeline, 
            verbose=1,
            callbacks=callbacks_list)

Epoch 1/20


2022-10-30 17:45:30.050783: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2038431744 exceeds 10% of free system memory.
2022-10-30 17:45:30.052111: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2038431744 exceeds 10% of free system memory.


      1/Unknown - 77s 77s/step - loss: 0.6778 - binary_accuracy: 0.5574 - mse: 0.2470

2022-10-30 17:46:53.850345: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2038431744 exceeds 10% of free system memory.
2022-10-30 17:46:53.853210: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2038431744 exceeds 10% of free system memory.


      2/Unknown - 142s 65s/step - loss: 0.6348 - binary_accuracy: 0.6183 - mse: 0.2271

2022-10-30 17:47:57.671800: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2038431744 exceeds 10% of free system memory.
2022-10-30 17:47:57.675174: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2038431744 exceeds 10% of free system memory.


     24/Unknown - 1504s 62s/step - loss: 0.2095 - binary_accuracy: 0.9425 - mse: 0.0603

In [ ]:
import matplotlib.pyplot as plt

x,y = next(iter(train_data_pipeline))
idx = 1
depth = 32

im = x[idx, depth]
lbl = y[idx, depth]
pred = sim_model.predict(x)[idx, depth].squeeze()

fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(im)
ax[1].imshow(lbl, cmap='Reds')
ax[2].imshow(pred, cmap='Greens')
plt.show()